In [180]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm, metrics
from tqdm import tqdm_notebook as tqdm
import random

In [2]:
path_to_data = "../data/train_images.npy"
path_to_labels = "../data/train_labels.csv"
path_to_test = "../data/test_images.npy"

In [128]:
# data = np.load(path_to_data, encoding = 'bytes')
labels_df = pd.read_csv(path_to_labels)
labels_df.Category = pd.Categorical(labels_df.Category)
y = labels_df.Category.cat.codes.values

In [80]:
def plot_images(list_of_images, max_col = 4):
    n = len(list_of_images)
    if n == 1:
        plt.imshow(list_of_images[0], cmap = 'gray_r'); plt.axis('off'); plt.show()
    else:
        # get number of columns and rows required
        r, c = 1, n
        if n > max_col:
            c = max_col
            r = int(math.ceil(n/max_col))
    
        fig = plt.figure(figsize=(17, max_col * r))
        for i, (img,name) in enumerate(list_of_images):
            ax = fig.add_subplot(r, c, (i+1))
            ax.set_title(str(name))
            ax.axis('off')
            ax.imshow(img, cmap = 'gray_r')

In [142]:
X = list(data[:, 1])

In [149]:
x_tr, x_val, y_tr, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

In [185]:
lSVM_cs = [1,10000]
n_fine_tune = 50

In [170]:
from sklearn import svm
def linear_SVM(x_tr, y_tr, x_val, y_val):
    f1_scores = []
    model = svm.LinearSVC(max_iter=100000)
    model.fit(x_tr, y_tr)
    preds = model.predict(x_val)
    cur_f1 = metrics.f1_score(y_val, preds, average = 'micro')

    return model, cur_f1

In [188]:
from sklearn import svm
def fine_tune_linear_SVM(x_tr, y_tr, x_val, y_val,
                         number_of_model, cs):
    all_c = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

    
    all_c = [c/100 for c in all_c]
    f1_scores = []
    print(all_c)
    best_model = None
    for c in tqdm(all_c):
        model = svm.LinearSVC(C=c, max_iter=1000000) # max_iter is a very large number so there is convergence
        model.fit(x_tr, y_tr)
        preds = model.predict(x_val)
        cur_f1 = metrics.f1_score(y_val, preds, average = 'micro')
        f1_scores.append(cur_f1)
        if cur_f1 == max(f1_scores):
            best_model = model
    print("The values of Cs considered are:", all_c)
    print("The best f1 score is:", max(f1_scores))
    print("The c is set to:", best_model.get_params()['C'])
    return best_model

In [ ]:
svm = fine_tune_linear_SVM(x_tr, y_tr, x_val, y_val, n_fine_tune, lSVM_cs)

[1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1, 1.0]


HBox(children=(IntProgress(value=0, max=7), HTML(value='')))

In [169]:
svm

(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=100000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0), 0.041212121212121214)